In [4]:
import numpy as np

## Part 1

# Batch mode training using least squares - supervised learning of network weights. 


- Implement a radial basis function network from scratch. 

- The network will be used to approximate sin(2x) and square(2x) functions




In [5]:
## Support functions for the evaluation

def getTrainSet(func = 'sin2x', stepSize = 0.1):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    train = np.zeros((2,N))
    inputs = np.arange(0, N)
    np.random.shuffle(inputs)
    
    for step, i in enumerate(inputs):
        train[0, i] = step*stepSize # Input: will be for example 0, 0.1, 0.2 .. 2pi etc.. 
        if (func == 'sin2x'): 
            train[1, i] = np.sin(2*step*stepSize) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            train[1, i] = np.sign(np.sin(2*step*stepSize)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return train.T

def getTestSet(func = 'sin2x', stepSize = 0.1):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    test = np.zeros((2,N))

    for step in range(N):
        test[0, step] = step*stepSize+0.05 # Input: will be for example 0.05, 0.15, 0.25 .. 2pi´0.05 etc.. 
        if (func == 'sin2x'): 
            test[1, step] = np.sin(2*step*stepSize+0.05) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            test[1, step] = np.sign(np.sin(2*step*stepSize+0.05)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return test.T

In [6]:
class RBF:
    def __init__(self, n = 12, variance = 0.1, maxinput = 6.28):
        self.n = n # the number of nodes
        self.variance = 0.1 ## Same variance for all nodes
        # self.units = np.random.rand(1, n) * maxinput # random unit position in the input space
        # self.units = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]) # unit positions 
        
        self.units = np.arange(0, maxinput, maxinput/(n-1))
        self.units = np.append(self.units, maxinput)
        self.n = self.units.shape[0]
        self.w = np.random.rand(1,n)
    
    def error(self, f_approx, f):
        if not (f_approx.shape == f.shape): 
            raise Exception('f_approx and f shapes mismatch. f_approx: {}, f: {} '.format(f_approx.shape, f.shape))

        # error = (phi(x) - target)^2
        return np.average(np.abs(f_approx - f))
    
    def predict(self, inp):
        # takes an input inp and returns predictions
        # do f^ = phi(x) * w.T
        x = inp.dot(np.ones((1, self.n))) # inp x 1 * 1 x n --> inp x n
        
        # m x n * n x 1 --> m x 1
        # print("m x n:")
        # print(x.shape)
        print("n x 1:")
        print(self.w.shape)
        f_approx = self.phi_matrix(x).dot(self.w)
        # print("output shape: " + str(f_approx.shape))
        return f_approx

    
    def fit(self, inp, f):
        # x is the input. Shape: inputs x 1
        # f is the true value of the function (aka the target), same shape
        if not (inp.shape == f.shape): 
            raise Exception('inp and f shapes mismatch. inp: {}, f: {} '.format(x.shape, f.shape))
        
        # print("input shape (m x 1):" + str(inp.shape))
        # print("target shape (m x 1):" + str(f.shape))

        # We want x to be a matrix with shape: inputs x neurons
        x = inp.dot(np.ones((1, self.n))) # inp x 1 * 1 x n --> inp x n
        
        # print("creted x with shape " + str(x.shape))
        
        # get phi(x)
        self.phi_x = self.phi_matrix(x)
        # print(self.phi_x.shape)

        # we obtain the w that minimizes the error by solving: 
        # phi(x).T * phi(x) * w = phi(x).T * f
        # --> w = (phi(x).T * phi(x))^-1 * phi(x).T * f
        # print(f.shape)
        
        self.w = np.linalg.inv(self.phi_x.T.dot(self.phi_x)).dot(self.phi_x.T).dot(f)
        
        print('Model fitted')
    
    def phi(self, x, i): 
        return np.exp(-(np.square(x-i))/(2*np.square(self.variance)))
    
    def phi_matrix(self, x):
        # number of inputs (m) (rows) x self.n (n) (columns)
        print("phi_matrix input size:" + str(x.shape))
        # this is a slow and stupid way of computing phi(x)
        for m in range(x.shape[0]): # m
            for n in range(x.shape[1]): # n
                res = self.phi(x = x[m, n], i = self.units[n])
        #        print("phi({}, {}): {}".format(x[row, col], self.units[0, row], res))
                x[m, n] = res

        return x
    
    

In [7]:
import matplotlib.pyplot as plt
def test(n):     
    model = RBF(n, variance = 0.1)
    
    function = 'sin2x'
    
    # Train model
    trainset = getTrainSet(function)
    # print(function + ' train set example: ')
    # print(trainset[0:5, :])
    # print("Units: ")
    # print(model.units)
    model.fit(trainset[:, 0][np.newaxis,:].T, trainset[:, 1][np.newaxis,:].T)
    
    
    ## Test Model
    
    # Get test set
    testset = getTestSet(function)
    # Make predictions
    pred = model.predict(testset[:, 0][np.newaxis,:].T)
    # test error
    e = model.error(pred, testset[:, 1][np.newaxis,:].T)
    # 
    # print("predictions")
    # print(pred[0:5, :])
    # print("target")
    # print(testset[0:5, :])
    # 
    # print("Absolute residual error: {}".format(e))
    # 
    return e

# nvars = np.array([10, 20, 30, 40, 50, 60, 61, 62, 63])
# fvars = np.ones(nvars.shape)
# for i, n in enumerate(nvars):
#     fvars[i] = test(n)
#  

test(10)


## Scatter Plot
fig = plt.figure()
ax = fig.add_subplot(211)
ax.scatter(nvars, fvars, marker = "o")
ax.set(title='',
        xlabel = "neurons",
        ylabel="residual error",
      ylim = (0, 0.7))
plt.show()

phi_matrix input size:(62, 10)
Model fitted
n x 1:
(10, 1)
phi_matrix input size:(62, 10)


NameError: name 'nvars' is not defined

In [37]:
test(60)
model.phi_x


print(test(100))
model.phi_x

phi_matrix input size:(62, 60)
Model fitted
phi_matrix input size:(62, 60)
phi_matrix input size:(62, 100)
Model fitted
phi_matrix input size:(62, 100)
6.68954069422e+13


array([[  0.00000000e+000,   0.00000000e+000,   1.16947281e-310, ...,
          1.09988986e-075,   3.05628512e-089,   6.35250785e-104],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          1.65101631e-003,   1.41735335e-006,   9.10147076e-011],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          9.15380442e-001,   4.92754153e-001,   1.98410947e-002],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          8.45436095e-001,   9.09460783e-002,   7.31802419e-004],
       [  8.37894253e-126,   1.39300896e-109,   1.73230869e-094, ...,
          1.29815834e-275,   4.66271947e-301,   0.00000000e+000],
       [  1.26641655e-014,   1.36186389e-009,   1.09546246e-005, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000]])

### Minimizing the residual error

With 12 units evenly distributed, variance = 0.1, the Absolute residual error was: 0.32240
the error did not change significantly with variance 0.3 or 0.01


With 20 units evenly distributed, variance = 0.1, the Absolute residual error was: 0.131492


Adding a node in the very end of the interval reduced it somewhat to 0.12314

30 units: 0.03243

40 units: 0.0313 


#### Questions:

- What is the lower bound for the number of training examples, N?
N must be larger than the number of neurons, n. 

- What happens with the error if N = n? Why?
The error starts increasing. This is because the system is overdetermined. 
The model can only reach points lying in some fixed plane and can never exactly match y^ if it lies somewhere outside the plane

- Under what conditions, if any, does (4) have a solution in this case?
Yes, if some equation occurs several times in the system, or if some equations are linear combinations of the others. 


In [162]:
a = np.ones((62, 10))
b = np.ones((10, 1))
a.

In [53]:
x = np.random.rand(10, 62)
x

array([[ 0.48019458,  0.81885275,  0.82930642,  0.44603195,  0.87760208,
         0.73910216,  0.28358219,  0.58970881,  0.65755535,  0.92755302,
         0.05690663,  0.83766092,  0.74423659,  0.60641523,  0.5977054 ,
         0.96429012,  0.34903727,  0.93972671,  0.75811998,  0.31139425,
         0.27641961,  0.69959149,  0.61484856,  0.66602187,  0.55072389,
         0.99419908,  0.35721938,  0.31176724,  0.7327584 ,  0.80676458,
         0.52314993,  0.78071668,  0.99320792,  0.10338771,  0.003168  ,
         0.39117372,  0.85282454,  0.60804737,  0.74130174,  0.14879854,
         0.52707445,  0.87456245,  0.00739324,  0.89133492,  0.85228835,
         0.54774499,  0.08255565,  0.9190222 ,  0.63096524,  0.02035962,
         0.02691458,  0.38703406,  0.24136002,  0.43103215,  0.67393478,
         0.10272357,  0.168787  ,  0.08071918,  0.75065871,  0.2660129 ,
         0.09216751,  0.41623994],
       [ 0.94281116,  0.68134357,  0.52849469,  0.68262971,  0.03985776,
         0.54789

In [77]:
y = np.random.rand(2, 3)
y

array([[ 0.73250323,  0.85098012,  0.66279403],
       [ 0.32049406,  0.39761901,  0.61799031]])

In [69]:
np.ones((1, 12)).shape

(1, 12)